In [ ]:
!pip install langgraph langchain

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
def model(APIKEY):

    return InferenceClient(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        token=APIKEY
    )

client=model("hf_tsgvbdlakn...")   # Enter your HuggingFace API here...

In [ ]:
def Generate(role, message,max_tokens=50):
    system_prompt=f"You are a {role}. Respond to the topic in your own perspective."
    response=client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message},
        ],
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content.strip()

In [ ]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda

def run_debate(topic, rounds=4):
    print(f"\n\nEnter topic for debate: {topic}")
    print("Starting debate between Scientist and Philosopher...\n")

    history=[]
    last_msg=f"The debate topic is: {topic}"

    scientist_node=RunnableLambda(lambda x: Generate("scientist", x).split('\n')[0])
    philosopher_node=RunnableLambda(lambda x: Generate("philosopher", x).split('\n')[0])
    judge_node=RunnableLambda(lambda x: Generate("judge", x, max_tokens=200))

    graph=StateGraph(dict)

    graph.add_node("Scientist", scientist_node)
    graph.add_node("Philosopher", philosopher_node)

    graph.set_entry_point("Scientist")
    graph.add_edge("Scientist", "Philosopher")
    graph.add_edge("Philosopher", "Scientist")

    debate_graph=graph.compile()

    for i in range(rounds):
        print('---' * 40)
        print(f"[Round {i+1}]\n")

        scientist_msg=scientist_node.invoke(last_msg)
        print(f"-> Scientist: {scientist_msg}\n")
        history.append(("Scientist", scientist_msg))

        philosopher_msg=philosopher_node.invoke(scientist_msg)
        print(f"-> Philosopher: {philosopher_msg}\n")
        history.append(("Philosopher", philosopher_msg))

        last_msg=philosopher_msg

    print('---' * 40 + "\n")
    debate_transcript="\n".join([f"{role}: {msg}" for role, msg in history])
    judge_msg=judge_node.invoke(f"Summarize and decide the winner of the following debate:\n{debate_transcript}")
    print(f"[Judge] Summary of debate:\n\n{judge_msg}")

In [ ]:
run_debate("Should AI be regulated like medicine?")



Enter topic for debate: Should AI be regulated like medicine?
Starting debate between Scientist and Philosopher...

------------------------------------------------------------------------------------------------------------------------
[Round 1]



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


-> Scientist: As a scientist, I firmly believe that regulating AI in a similar manner to medicine is a crucial step towards ensuring its safe and responsible development and deployment. Here's why:

-> Philosopher: A fascinating convergence of philosophical and scientific perspectives.  As a philosopher, I must say that I both agree and disagree with your stance on regulating AI in a similar manner to medicine. 

------------------------------------------------------------------------------------------------------------------------
[Round 2]

-> Scientist: I'm delighted to engage in a discussion that bridges the realms of philosophy and science.  As a scientist, I've long been intrigued by the parallels between the development of artificial intelligence (AI) and the evolution of medical practices.

-> Philosopher: What a fascinating intersection of two disciplines that often seem worlds apart.  The parallels you've drawn between the development of artificial intelligence and the evolut